In [ ]:
# bootstrap environment into place
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import os
from googleapiclient.http import MediaIoBaseDownload

def download_file(fn, file_id):
    request = drive_service.files().get_media(fileId=file_id)
    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
    done = False
    while done is False:
        # _ is a placeholder for a progress object that we ignore.
        # (Our file is small, so we skip reporting progress.)
        _, done = downloader.next_chunk()
    
    downloaded.seek(0)

    folder = fn.split('/')
    if len(folder) > 1:
        os.makedirs(folder[0], exist_ok=True)

    with open(fn, 'wb') as f:
        f.write(downloaded.read())

id_to_fn = {
        '1-1Ehz1QBrjYqfH6SITCbmn6B29CyCPqo': "data/images.npz",
        '1-OWMg8uy2bvuONw_0mmCMaEJBmckvxud': "data/train_data.npz",
        '1-GNpWqe4T1UTWY_QnvzJDU7-WCc1uDIL': "data/valid_data.npz",
        '1-Wk9OoipexkS8BlVR_1_TD3Ay1vgzAl-': "data/kpt.npy",
        '1-R8Ml635hqPaNNDhNUDspEZZJW2AzsnG': "data/misc_data.npz",
        '1-cXm7Q1m7pZQYlUKpCHchHCwbmQDV299': "utils/__init__.py",
        '1-c3vs68eYuBrdaj0Emim46aGVz4xdeUW': "utils/pose_utils.py",
        '1-YX63uppCMtOwxXfQta9z97AihtWQpY8': "utils/plotting.py",
        '1-bViGma06XmSbJGYddfeofFP8LXqi1bQ': "utils/train_options.py",
        '1-8vm2jmtbGpquuw-FHuaBRTz2P7ItrMb': "hourglass.py",
        '1-Pdkc9DMm-d1eJaOuXtJTXrgFsqZCE1W': "dataset.py",
           }

# download all files into the vm
for fid, fn in id_to_fn.items():
    download_file(fn, fid)

# mount the user's google drive into the system
from google.colab import drive
drive.mount('/content/gdrive')
# create folder to write data to
os.makedirs('/content/gdrive/My Drive/cis580-2020-hw8', exist_ok=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from __future__ import division
from __future__ import print_function

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid

from hourglass import hg

from dataset import Dataset
from dataset import RandomFlipLR, RandomRescaleBB, RandomRotation,\
                    CropAndPad, LocsToHeatmaps,\
                    ToTensor, Normalize, Denormalize
from utils import TrainOptions, PoseUtils

from tqdm import tqdm
from PIL import Image
import os
from time import time
import numpy as np


In [ ]:
%matplotlib inline


import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
from torch.utils.tensorboard import SummaryWriter
import tqdm
print("Version:", torch.__version__)
print("Cuda available:", torch.cuda.is_available())

Version: 1.4.0
Cuda available: True


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
class Trainer(object):

    def __init__(self, options):
        self.options = options
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        train_transform_list = []
        train_transform_list.append(RandomRescaleBB(0.8, 1.2))
        train_transform_list.append(RandomFlipLR())
        train_transform_list.append(RandomRotation(degrees=options.degrees))
        train_transform_list.append(CropAndPad(out_size=(options.crop_size, options.crop_size)))
        train_transform_list.append(LocsToHeatmaps(out_size=(options.heatmap_size, options.heatmap_size)))
        train_transform_list.append(ToTensor())
        train_transform_list.append(Normalize())
        test_transform_list = [CropAndPad(out_size=(options.crop_size, options.crop_size))]
        test_transform_list.append(LocsToHeatmaps(out_size=(options.heatmap_size, options.heatmap_size)))
        test_transform_list.append(ToTensor())
        test_transform_list.append(Normalize())
        self.train_ds = Dataset(is_train=True, transform=transforms.Compose(train_transform_list))
        self.test_ds = Dataset(is_train=False, transform=transforms.Compose(test_transform_list))

        self.model = hg(num_stacks=options.num_stacks, num_blocks=options.num_blocks, num_classes=options.num_classes).to(self.device)
        # define loss function and optimizer
        self.heatmap_loss = torch.nn.MSELoss().to(self.device) # for Global loss
        self.optimizer = torch.optim.RMSprop(self.model.parameters(),
                                             lr = options.lr)


        self.pose_utils = PoseUtils(detection_thresh=options.detection_thresh, dist_thresh=options.dist_thresh)

        self.train_data_loader = DataLoader(self.train_ds, batch_size=options.batch_size,
                                            num_workers=options.num_workers,
                                            pin_memory=True,
                                            shuffle=True)
        self.test_data_loader = DataLoader(self.test_ds, batch_size=options.test_batch_size,
                                           num_workers=options.num_workers,
                                           pin_memory=True,
                                           shuffle=True)

        self.summary_iters = []
        self.losses = []
        self.pcks = []

    def train(self):
        self.total_step_count = 0
        start_time = time()
        for epoch in range(1,self.options.num_epochs+1):
            elapsed_time = time() - start_time
            seconds_per_epoch = elapsed_time / epoch
            remaining_time = (self.options.num_epochs - epoch) * seconds_per_epoch

            print("Epoch %d/%d - Elapsed Time %f - Remaining Time %f" %
                    (epoch, self.options.num_epochs,
                        elapsed_time, remaining_time))

            for step, batch in enumerate(self.train_data_loader):
                self.model.train()
                batch = {k: v.to(self.device) if isinstance(v, torch.Tensor) else v for k,v in batch.items()}
              
                image,label=batch['image'],batch['keypoint_heatmaps']
                image, label = image.to(device), label.to(device)
                self.optimizer.zero_grad()
                image=image.view(-1,3,256,256)       #flattening the images
                pred_heatmap_list=self.model(image)
                loss = self.heatmap_loss(pred_heatmap_list[0],label)
                loss.backward()
                self.optimizer.step()
                self.total_step_count += 1
                if self.total_step_count % self.options.summary_steps == 0:
                    self.save_summaries(batch, pred_heatmap_list[-1].detach(), loss.detach(), mode='train')
                if self.total_step_count % self.options.test_steps == 0:
                    self.model.eval()
                    batch = next(iter(self.test_data_loader))
                    batch = {k: v.to(self.device) if isinstance(v, torch.Tensor) else v for k,v in batch.items()}
                 
                    with torch.no_grad(): 
                      image,label=batch['image'],batch['keypoint_heatmaps']
                      image, label = image.to(device), label.to(device)
                      self.optimizer.zero_grad()
                      image=image.view(-1,3,256,256)       #flattening the images
                      pred_heatmap_list=self.model(image)
                      loss = self.heatmap_loss(pred_heatmap_list[0],label)
                      self.save_summaries(batch, pred_heatmap_list[-1], loss, mode='test')

        checkpoint = {'model': self.model.state_dict()}
        torch.save(checkpoint, '/content/gdrive/My Drive/cis580-2020-hw8/model_checkpoint.pt')

    def save_summaries(self, batch, pred_heatmaps, loss, mode='train'):
        gt_heatmaps = batch['keypoint_heatmaps'].to(self.device)
        pck = self.pose_utils.pck(gt_heatmaps, pred_heatmaps)

        batch = Denormalize()(batch)
        images = batch['image']

        gt_image_keypoints, pred_image_keypoints = self.pose_utils.draw_keypoints_with_labels(images, gt_heatmaps, pred_heatmaps)

        gt_image_keypoints_grid = make_grid(gt_image_keypoints, pad_value=1, nrow=3)
        pred_image_keypoints_grid = make_grid(pred_image_keypoints, pad_value=1, nrow=3)

        pred_heatmaps_grid = make_grid(pred_heatmaps[0,:,:,:].unsqueeze(0).transpose(0,1), pad_value=1, nrow=5)
        pred_heatmaps_grid[pred_heatmaps_grid > 1] = 1
        pred_heatmaps_grid[pred_heatmaps_grid < 0] = 0

        self.losses.append(loss)
        self.pcks.append(pck)
        self.summary_iters.append(self.total_step_count)

        folder = '/content/gdrive/My Drive/cis580-2020-hw8/logs/%d' % self.total_step_count
        os.makedirs(folder, exist_ok=True)

        gt_ikg = Image.fromarray((gt_image_keypoints_grid.cpu().detach().numpy().transpose(1,2,0) * 255.).astype(np.uint8))
        pred_ikg = Image.fromarray((pred_image_keypoints_grid.cpu().detach().numpy().transpose(1,2,0) * 255.).astype(np.uint8))
        pred_hg = Image.fromarray((pred_heatmaps_grid.cpu().detach().numpy().transpose(1,2,0) * 255.).astype(np.uint8))

        gt_ikg.save(os.path.join(folder, mode + '_gt_image_keypoints.jpg'))
        pred_ikg.save(os.path.join(folder, mode + '_pred_image_keypoints.jpg'))
        pred_hg.save(os.path.join(folder, mode + '_pred_heatmaps.jpg'))

In [ ]:
options = TrainOptions().parse_args(["--name=gascan",
                                     "--summary_steps=200",
                                     "--num_epochs=400",
                                     "--test_steps=1000"])
trainer = Trainer(options)
trainer.train()

Epoch 1/400 - Elapsed Time 0.000004 - Remaining Time 0.001522
Epoch 2/400 - Elapsed Time 4.448466 - Remaining Time 885.244794
Epoch 3/400 - Elapsed Time 8.429119 - Remaining Time 1115.453429
Epoch 4/400 - Elapsed Time 12.397531 - Remaining Time 1227.355525
Epoch 5/400 - Elapsed Time 16.535113 - Remaining Time 1306.273953
Epoch 6/400 - Elapsed Time 20.567053 - Remaining Time 1350.569803
Epoch 7/400 - Elapsed Time 24.635144 - Remaining Time 1383.087383
Epoch 8/400 - Elapsed Time 28.663676 - Remaining Time 1404.520137
Epoch 9/400 - Elapsed Time 32.744880 - Remaining Time 1422.583139
Epoch 10/400 - Elapsed Time 36.814355 - Remaining Time 1435.759841
Epoch 11/400 - Elapsed Time 40.884495 - Remaining Time 1445.824406
Epoch 12/400 - Elapsed Time 46.682902 - Remaining Time 1509.413834
Epoch 13/400 - Elapsed Time 50.793683 - Remaining Time 1512.088872
Epoch 14/400 - Elapsed Time 54.807471 - Remaining Time 1511.120266
Epoch 15/400 - Elapsed Time 58.941510 - Remaining Time 1512.832089
Epoch 16/40

The next block will run prediction on the test set and save the output detections

In [ ]:
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

from hourglass import hg

from dataset import Dataset
from dataset import CropAndPad, LocsToHeatmaps, ToTensor, Normalize
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = hg(num_stacks=1, num_blocks=1, num_classes=10).to(device)
checkpoint = torch.load('/content/gdrive/My Drive/cis580-2020-hw8/model_checkpoint.pt')
model.load_state_dict(checkpoint['model'])
transform_list = [CropAndPad(out_size=(256, 256))]
transform_list.append(LocsToHeatmaps(out_size=(64,64)))
transform_list.append(ToTensor())
transform_list.append(Normalize())
test_ds = Dataset(is_train=False, transform=transforms.Compose(transform_list))
data_loader = DataLoader(test_ds, batch_size=1, pin_memory=True, shuffle=False)
predicted_heatmaps = np.zeros((len(test_ds), 10, 64, 64))
for i, batch in enumerate(tqdm(data_loader)):
    model.eval()
    images = batch['image'].to(device)
    with torch.no_grad():
        pred_heatmap_list = model(images)
    pred_heatmaps = pred_heatmap_list[-1]
    predicted_heatmaps[i,:,:,:] = pred_heatmaps[0,:,:,:].cpu().numpy()
np.save('/content/gdrive/My Drive/cis580-2020-hw8/detections.npy', predicted_heatmaps)

100%|██████████| 27/27 [00:01<00:00, 26.11it/s]


Import the external and given functions for step 2

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import cv2
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
from os.path import join
from utils import plot_mesh

class Rodrigues(torch.autograd.Function):
    @staticmethod
    def forward(self, inp):
        pose = inp.detach().cpu().numpy()
        rotm, part_jacob = cv2.Rodrigues(pose)
        self.jacob = torch.Tensor(np.transpose(part_jacob)).contiguous()
        rotation_matrix = torch.Tensor(rotm.ravel())
        return rotation_matrix.view(3,3)

    @staticmethod
    def backward(self, grad_output):
        grad_output = grad_output.view(1,-1)
        grad_input = torch.mm(grad_output, self.jacob)
        grad_input = grad_input.view(-1)
        return grad_input

rodrigues = Rodrigues.apply

In [ ]:

def pose_optimization(img, vertices, faces, keypoints_2d, conf, keypoints_3d, K):
    # Send variables to GPU
    device = keypoints_2d.device
    keypoints_3d = keypoints_3d.to(device)
    K = K.to(device)
    r = torch.rand(3, requires_grad=True, device=device) # rotation in axis-angle representation
    t = torch.rand(3 ,requires_grad=True, device=device)
    d = conf.sqrt()[:, None]
    # 2D keypoints in normalized coordinates
    norm_keypoints_2d = torch.matmul(K.inverse(), torch.cat((keypoints_2d, torch.ones(keypoints_2d.shape[0],1, device=device)), dim=-1).t()).t()[:,:-1]
    # set up optimizer
    keypoints_3d=keypoints_3d.T
    D=torch.zeros(10,10)
    for i in range(10):
      D[i][i]=d[i]
    norm_keypoints_2d=norm_keypoints_2d.T
    optimizer = torch.optim.Adam([r,t], lr=1e-2)
    # converge check
    converged = False
    rel_tol = 1e-5
    loss_old = 100
    loss=torch.zeros(2,10)
    x=torch.zeros(3,10)
    proj_keypoints=torch.zeros(2,10)
    proj_loss =torch.zeros(2,10)
    while not converged:

      R = rodrigues(r)

      x = torch.mm(R, keypoints_3d)
      for i in range(10):
        x[:,i]=x[:,i]+t
      proj_loss=proj_keypoints
      proj_loss[0]=x[0]/x[2]
      proj_loss[1]=x[1]/x[2]
      optimizer.zero_grad()
      a=torch.matmul((proj_loss-norm_keypoints_2d),D)
      loss=(a**2).mean()
      loss.backward(retain_graph=True)
      optimizer.step()
      if(np.abs(loss.item() - loss_old) < rel_tol):
        converged=True
      print(np.abs(loss.item() - loss_old),converged)
      loss_old=loss.item()
    proj_keypoints=proj_keypoints.T
         
    R = rodrigues(r)
    plt.figure()
    plot_mesh(img, vertices, faces, R.detach().cpu().numpy(), t.detach().cpu().numpy()[:,None], K.detach().cpu().numpy())
    pk = torch.matmul(K, torch.cat((proj_keypoints, torch.ones(proj_keypoints.shape[0], 1, device=device)), dim=-1).t()).t()
    pk = pk.detach().cpu().numpy()
    plt.scatter(pk[:,0], pk[:,1], color='r')
    plt.show()
    return rodrigues(r)[0].detach(), t.detach()


def heatmaps_to_locs(heatmaps):
  heatmaps_locs=torch.zeros(27,10,3)
  heatmaps_x=torch.zeros(64)
  c=torch.zeros(1)
  for i in range(27):
    for j in range(10):
      heatmaps_x,a=torch.max(heatmaps[i][j],dim=1)
      c,y=torch.max(heatmaps_x,dim=0)
      x=a[y]
      heatmaps_locs[i][j][0]=x
      heatmaps_locs[i][j][1]=y
      heatmaps_locs[i][j][2]=c
  return heatmaps_locs

In [ ]:
data = np.load('data/misc_data.npz')
K = torch.from_numpy(data['K']).float()
S = torch.from_numpy(np.load('data/kpt.npy'))
vertices = data['vertices']
faces = data['faces']

dataset = np.load('data/valid_data.npz')
imgnames = dataset['imgname']
scales = torch.from_numpy(dataset['scale']).float()[:,None]
centers = torch.from_numpy(dataset['center']).float()

heatmaps = torch.from_numpy(np.load('/content/gdrive/My Drive/cis580-2020-hw8/detections.npy'))
keypoint_locs = heatmaps_to_locs(heatmaps)

ul_corner = centers - scales * 100
keypoint_locs[:,:,:-1] *= scales[:,:,None]*200 / 64
keypoint_locs[:,:,:-1] += ul_corner[:,None,:]

images = np.load('data/images.npz')

os.makedirs('/content/gdrive/My Drive/cis580-2020-hw8/pose_optimization', exist_ok=True)
for i in range(len(heatmaps)):
    img_name = imgnames[i].decode('utf-8')
    print("Processing ", img_name)
    # Fixes incorrect naming scheme
    img = images[img_name]
    pose_optimization(img, vertices, faces, keypoint_locs[i,:,:-1], keypoint_locs[i,:,-1], S, K)
    plt.savefig('/content/gdrive/My Drive/cis580-2020-hw8/pose_optimization/%s' % img_name, dpi=300)
    plt.close('all')

Streaming output truncated to the last 5000 lines.
0.00036728382110595703 False
0.0003669261932373047 False
0.00036656856536865234 False
0.0003663301467895508 False
0.00036585330963134766 False
0.000365525484085083 False
0.00036516785621643066 False
0.0003648698329925537 False
0.000364452600479126 False
0.00036400556564331055 False
0.0003638267517089844 False
0.00036334991455078125 False
0.000363081693649292 False
0.00036257505416870117 False
0.000362396240234375 False
0.0003618896007537842 False
0.00036138296127319336 False
0.0003610551357269287 False
0.00036081671714782715 False
0.00036025047302246094 False
0.0003599822521209717 False
0.0003597736358642578 False
0.0003593266010284424 False
0.00035893917083740234 False
0.0003586113452911377 False
0.00035825371742248535 False
0.00035768747329711914 False
0.00035756826400756836 False
0.00035691261291503906 False
0.00035685300827026367 False
0.00035640597343444824 False
0.0003559887409210205 False
0.00035569071769714355 False
0.000355184

KeyboardInterrupt: ignored